In [1]:
import pennylane as qml
from pennylane import numpy as np

## G.1.1

Concept: `qml.broadcast(one_wire_gate, wires=ws, pattern="single")`

In [3]:
n_bits = 4
dev = qml.device("default.qubit", wires=n_bits)


def oracle_matrix(combo):
    """Return the oracle matrix for a secret combination.

    Args:
        combo (list[int]): A list of bits representing a secret combination.

    Returns:
        array[float]: The matrix representation of the oracle.
    """
    index = np.ravel_multi_index(combo, [2] * len(combo))  # Index of solution
    my_array = np.identity(2 ** len(combo))  # Create the identity matrix
    my_array[index, index] = -1
    return my_array


@qml.qnode(dev)
def oracle_amp(combo):
    """Prepare the uniform superposition and apply the oracle.

    Args:
        combo (list[int]): A list of bits representing the secret combination.

    Returns:
        array[complex]: The quantum state (amplitudes) after applying the oracle.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.broadcast(qml.Hadamard, wires=range(n_bits), pattern="single")
    qml.QubitUnitary(oracle_matrix(combo), wires=range(n_bits))
    return qml.state()


## G.1.2a

In [ ]:
n_bits = 4


def diffusion_matrix():
    """Return the diffusion matrix.

    Returns:
        array[float]: The matrix representation of the diffusion operator.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    l = 2 ** n_bits
    my_array = 2 ** (1-n_bits) * np.ones(l) - np.identity(l)
    return my_array


@qml.qnode(dev)
def difforacle_amp(combo):
    """Apply the oracle and diffusion matrix to the uniform superposition.

    Args:
        combo (list[int]): A list of bits representing the secret combination.

    Returns:
        array[complex]: The quantum state (amplitudes) after applying the oracle
        and diffusion.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.broadcast(qml.Hadamard, wires=range(n_bits), pattern="single")
    qml.QubitUnitary(oracle_matrix(combo), wires=range(n_bits))
    qml.QubitUnitary(diffusion_matrix(), wires=range(n_bits))
    return qml.state()


## G.1.2b

In [ ]:
@qml.qnode(dev)
def two_difforacle_amp(combo):
    """Apply the Grover operator twice to the uniform superposition.

    Args:
        combo (list[int]): A list of bits representing the secret combination.

    Returns:
        array[complex]: The resulting quantum state.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.broadcast(qml.Hadamard, wires=range(n_bits), pattern="single")
    qml.QubitUnitary(oracle_matrix(combo), wires=range(n_bits))
    qml.QubitUnitary(diffusion_matrix(), wires=range(n_bits))
    qml.QubitUnitary(oracle_matrix(combo), wires=range(n_bits))
    qml.QubitUnitary(diffusion_matrix(), wires=range(n_bits))
    return qml.state()
